# Step 0: Setup matlab engine

On your remote.esrf.fr, run the following commands in a shell:

## Step 0.1 Launch the kernel and select your experiment to post-treat

In [ ]:
try:
    from esrf_pathlib import ESRFPath as Path
except ImportError:
    from pathlib import Path
import esrf_statusgui.exp_methods.dct.matlab_engine as mleng 
ml = mleng.MatlabConnector()
dataset = Path('')

## Step 0.2: To continue with the setup and if restarting a project

In [2]:
ml.cd(f'{str(dataset)}')

In [ ]:
ml.eval("p=gtLoadParameters()")

# Step 1: Apply image distortions corrections and create the .edf files

Run all the following cells to launch the setup of the RAW data.

In [ ]:
ml.eval("gtSaveParameters(p)")
ml.eval(f"gtLaunchPreprocessing(p, 4);")

# Step 2: Preprocessing

Step 2: image preprocessing

Image processing includes: background correction by rolling median subtraction for diffracted beam area, cropping direct beam area for absorption tomography reconstruction and flat field correction for these cropped images.

Normally it takes ~5 min.


In [ ]:
ml.gtPreprocessing()

# Step 3: spot segmentation

The next part is about segmentation of the images. 

Two types of outputs are genrated in the spot segmentation:

1. Difblobs: In the case of intragranular misorientation, the diffraction of each grain can occur in a range of the sample rotation angle. 
A blob corresponds to segmented spots connected over several pictures corresponding to a sample rotation range (omega).
2. Difspots: Sum of the blobs along the omega axis

Follow this link for detailed tutorial:

In [ ]:
ml.gtSegmentationGUI()


# Step 4: Friedel pair spots matching


Pre-match, based on spot locations, shapes, intensities and bounding boxes

tune position X, Y, tilts to fit the geomtry, make the distribution of spots locate on the predicted 'lines'

Match (matching parameters can be tuned, e.g. tolerance), 'save' clicke multiple time to not lost the parameters saving

plot - pair figure: view the Friedel pair spots

reset -> match, to match Friedel pairs with the updated geometry

In [ ]:
ml.gtMATCHGUI()

# Step 5: gtSetupIndexing
results written to 4_grains/phase01/index.mat

In [ ]:
ml.gtSetupIndexing(nargout=0)

# Optional Step 5 bis: Fit the energy and reflect the energy change onto the lattice parameter

In [ ]:
ml.gtFitEnergyToLatticepar(nargout=0)

Run this cell if you see a mismatch between the expected and observed reflections. Instead of changing the energy (to preserve any crossed-technique analysis e.g. with FF data, the shift of energy is applied to the lattice parameter. This is only compatible with 1 phase analysis up to now.

# Step 6.0: Globalfit

In [ ]:
ml.eval("p=gtLoadParameters();")
# ml.eval("p=rmfield(p, 'globalfit')")
ml.eval("[p, opt]=gtRunGlobalFit(p, 'do_plot', 0)")

If you are doing this step for the first time. You should go back to gtMATCHGUI to take into account the geometry optimisation. Run all the steps from gtMATCHGUI.

# Step 6: gtSetupForwardSimulation

to pair the spots to the indexed orientation


In [ ]:
ml.gtSetupForwardSimulation()

# Step 7: gtSetupReconstruction

## 7.1 ASTRA toolbox for reconstructing grain shapes

In [ ]:
ml.gtSetupReconstruction()

check p.rec.absorption: padding => helps if the sample is out of FOV in some projections

step 1: reconstruct and segmentate volume mask (this step has been skipped in the new code running on SLURM cluster)

step 2: grain reconstruction (based on SIRT algorithm by default using ASTRA)

step 3: grain segmentation, to remove noise of the reconstructed grains (automatic now on SLURM cluster)

View grain information, e.g.


## 7.2 Visualization

In [ ]:
ml.eval("h = GtAssembleVol3D;")
ml.eval("h.localPar.deal_with_twins = true;")
ml.eval("h.assembleAllPhases;")
ml.eval("h.assembleCompleteVolume;")
ml.eval("h.dilateCompleteVolume(1);")
ml.eval("sample = GtSample.loadFromFile;")
ml.eval("sample.dilVolFile = '5_reconstruction/volume_dilated.mat';")
ml.eval("sample.saveToFile;")

In [ ]:
ml.GtGrainsManager(nargout=0)

Visualize only one grain ?

In [ ]:
ml.eval("g1 = gtLoadGrain(1, 1)")

ml.eval('GtVolView(g1.proj.stack)')

ml.gtShowFsim(1, 1)

## 7.3 6D reconstruction ?

By default, the reconstruction algorithm is SIRT, you can view the algorithm settings from parameters.rec.grains

If you want to do 6D reconstruction, i.e. reconstruct not only the grain shape with average orientation but also reconstruct voxel orientations within each grain, you can modify the algorithm:
    
% change algorithm from SIRT to 6DTV


In [ ]:
ml.eval("p=gtLoadParameters;")
ml.eval("p.rec=gtRecDefaultParameters('grains_algo', '6DTV');")
    
# % set parameters for 6DTV, e.g.
ml.eval("p.rec.grains.options.ospace_resolution = 0.1;")
ml.eval("p.rec.grains.options.use_predicted_scatter_ints = 1;")
ml.eval("p.rec.grains.options.use_matrix_row_rescaling = 1;")
    
# % save parameters
ml.eval("gtSaveParameters(p);")
    
# % launch grain reconstruction, gtSetupReconstruction(phase_id, skip_flag), e.g.
# % launch 6D reconstruction for all grains
ml.eval("gtSetupReconstruction(1,1);")


In [ ]:
ml.eval("h = GtAssembleVol3D;")
ml.eval("h.localPar.deal_with_twins = true;")
ml.eval("h.assembleAllPhases;")
ml.eval("h.assembleCompleteVolume;")
ml.eval("h.dilateCompleteVolume(1);")
ml.eval("sample = GtSample.loadFromFile;")
ml.eval("sample.dilVolFile = '5_reconstruction/volume_dilated.mat';")
ml.eval("sample.saveToFile;")
# % launch 6D reconstruction for a grain list, e.g. [1 2 3]
# ml.eval("gtSetupReconstruction(1,1,'grain_ids',[1 2 3])")
    
# % for one grain running on local gpu machine, e.g. phase_id = 1 and grain_id = 120
# ml.eval("grec=gtReconstructGrainOrientation(120, 1, p, 'save', true);")

## 7.4 Filter grains

In [ ]:
ml.eval("sample = GtSample.loadFromFile()")
ml.eval("[sample, bad, bad_solidity, bad_completeness, bad_numspots, previous] = gtFilterGrains(sample, [], [])")

# Step 8 Automatic post-processing
Automatic processing for one single dataset

Process the above step 1-7 in one go for a single dataset without human intervention, i.e. you do not want to change anything with the parameter file !


## 8.1 Automate the setupH5 process (step 1)

In [ ]:
try:
    from esrf_loadFile import loadFile
except ImportError:
    from esrf_statusgui.data_managment.loadFile import loadFile
ml.eval('p=gtLoadParameters;')
# Automatic detection of the other folders
parameters = loadFile(dataset / 'parameters.h5')
raw_folder = Path(parameters.get_value('acq/collection_dir_old')).parent
exp_name = parameters.get_value('acq/name')
if 'dct' in exp_name:
    exp_name_done = exp_name.split('dct')[1]
else:
    exp_name_done = exp_name.split('_')[-1]
exp_name_common = exp_name[:-len(exp_name_done)]
other_datasets_ids = sorted([int(folder.name.split(exp_name_common)[1]) for folder in raw_folder.iterdir() if folder.is_dir() and exp_name_common in folder.name and dataset.name not in folder.name and (folder.name.split(exp_name_common)[1]).isdigit()])
# Or manual selections of the datasets to post-treat
# other_datasets_ids = [1, 2, 3]
print(f"gtBatchSetupH5(p, '{str(dataset.name[:-1])}', {str(other_datasets_ids)})")

Check if the call is correct (datasets indices from RAW_DATA).
Run the following line if okay.

In [ ]:
ml.eval(f"gtBatchSetupH5(p, '{str(dataset.name[:-1])}', {str(other_datasets_ids)})")

## 8.2 Automate the steps 2 to 7

In [ ]:
try:
    from esrf_loadFile import loadFile
except ImportError:
    from esrf_statusgui.data_managment.loadFile import loadFile
ml.eval('p=gtLoadParameters;')
# Automatic detection of the other folders
parameters = loadFile(dataset / 'parameters.h5')
exp_name = parameters.get_value('acq/name')
if 'dct' in exp_name:
    exp_name_done = exp_name.split('dct')[1]
else:
    exp_name_done = exp_name.split('_')[-1]
exp_name_common = exp_name[:-len(exp_name_done)]
other_datasets_ids = sorted([int(folder.name.split(exp_name_common)[1]) for folder in raw_folder.iterdir() if folder.is_dir() and exp_name_common in folder.name and dataset.name not in folder.name and (folder.name.split(exp_name_common)[1]).isdigit()])
# Or manual selections of the datasets to post-treat
print(f"gtBatchProcessSeries('{str(dataset.parent)}', '{str(dataset.name[:-1])}', {str(other_datasets_ids)})")

Same Business here !

In [ ]:
ml.eval(f"gtBatchProcessSeries('{str(dataset.parent)}', '{str(dataset.name[:-1])}', {str(other_datasets_ids)})")

## 2) Post analysis


In [ ]:

ml.eval("p=gtLoadParameters;")
ml.eval("g110=gtLoadGrain(1,110)")      # => view grain info
ml.eval("g110=gtLoadGrainRec(1,110)")   # => view grain reconstruction setting


In [ ]:
ml.eval("GtVolView(g110.VOL3D.intensity)")
ml.eval("GtVolView(g110.SEG.seg)")

ml.eval("GtMultiVolView({g110.proj.bl(:).intm})")  # for 4D matrix visulization or cell


ml.eval("GtSlideShow('./0_rawdata/sample_dct_0003/sample_dct_*.edf')") #to visualize edf image


ml.eval("gtShowFsim(110,1)")           # Check forward simulation and spots for Friedel Pair


ml.eval("varargout = gt6DPlotReconstructionODF(phase_id, grain_id, varargin)")